# Fokker-Planck Equation III: Operator Splitting

This notebook investigates operator splitting methods for the Fokker Planck equation. Thus far in the notebook series, we have investigated the two terms in the FPE (Advection and Diffusion) separately, and in this notebook, we will investigate how these independent term-wise solutions can be used in combination to integrate the entire advection-diffusion equation, and discuss why it works, as well as the motivations and runtime savings afforded to the full implementation as a result.

We can write the 1D Fokker-Planck equation, in general, as 

$$ \partial_t p(x, t) = \partial_x[\mu(x, t) p(x, t)] + D \partial_{xx}^2 p(x, t) $$

Which, can be written more concisely in terms of the differential operator $\mathcal{L}$ (often referred to as the *generator*) as

$$ \partial_t p(x, t) = \mathcal{L} \cdot p(x, t) $$

where the Fokker-Planck operator $\mathcal{L}$ is defined as

$$ \mathcal{L} \equiv \left( \partial_x \mu(x, t) + D\partial_{xx}^2 \right) $$

Formally, the solution for $p(x, t)$ of the operator equation is simple (given an initial distribution $p(x, t_0)$), and simply written as

$$ p(x, t) = e^{\mathcal{L} \Delta t} p(x, t_0) $$

however, in practice, this solution is defined only in terms of its Taylor series, and is virtually impossible to evaluate in practice. However, if you consider the limit where only a small amount of time has passed ($\Delta t \ll 1$) then the exponential can be well approximted with its linear-order Taylor series as

$$ p(x, t) \approx p(x, t_0) + \Delta t \mathcal{L}\cdot p(x, t_0) $$

which formally aprpoximates the actual FPE in the $\Delta t \to 0$ limit, as we can re-arrange this equation to read

$$ \frac{p(x, t) - p(x, t_0)}{\Delta t} \approx \partial_t p(x, t) = \mathcal{L}p(x, t_0)$$

So, what does this get us? Well, first, for finite but small timesteps, this equation gives an approximate means of iteratively updating the initial probability (essentialy what we are doing with the advection and diffusion terms in isolation). However, there is a set of methods (bradly known as *operator splitting* methods) that can significantly simplify the calculation of this in practice.

Before delving into how operator splitting works in practice, its worth brining attantion to our prior analysis of the integration schemes for advection and diffusion alone. For the advective equation, the Lax-Wendroff method was sufficient to give high levels of stability, but gets complicated and cumbersome to apply to higher order equations like those involving diffusion. Conversely, a primary consideration for improving wall clock times in the diffusion equation was the fact that we were able to (often) make the simplification to the diffusion update that the coefficient $D$ was constant, and thus we were only required to invert the Crank-Nicolson matrices onces, and then use that inverse going forward (which provided a significant speedup---that improves with increasing $N$---upon inverting a matrix in each iteration). For many dynamic problems (virtually all instances of the Soluchowski equation that we will be interested in) the dynamics enter through the advective term, then it would be nice to still retain the ability to use the constant-$D$ speedup.  Herein lies the benefit of operator splitting methods.  Note that we can generally write the full Fokker-Planck equation as a summation over the advective $\mathcal{L}_{\rm A}$ and diffusive $\mathcal{L}_{\rm D}$ terms, and so the formal solution can be written as

$$ p(x, t) = e^{\mathcal{L}_{\rm A} + \mathcal{L}_{\rm D}}p(x, t_0) $$

Now, if the operators were themselves represented as simple numbers, then we would be able to split up the action of the operator on the initial probability into two contributions explicitlty and treat the operation of each term independently of the others. However, this is (unfortunately) not the case, and so such a procedure will not be correct. However, for small $\Delta t$ this is, in fact, valid.  Consider expanding the summation-version of the Fokker-Planck solution to linear order in $\Delta t$,

$$ p(x, t) \approx (1 + \mathcal{L}_{\rm A}\Delta t + \mathcal{L}_{\rm D}\Delta t + \mathcal{O}(\Delta t^2)) p(x, t_0) $$


